In [153]:
import os
import numpy as np
import shutil
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer
import bottleneck as bn

In [154]:
import vaecf_util as util
import vaecf as model
import vaecf_metric as metric
import vaecf_preprocess as preprocess
import vaecf_train as trainer
import vaecf_evaluate as evaluator

In [322]:
import imp
util = imp.reload(util)
preprocess = imp.reload(preprocess)
trainer = imp.reload(trainer)
evaluator = imp.reload(evaluator)
metric = imp.reload(metric)

In [323]:
### Pre-process data

In [324]:
MODEL_DIR = '../model/'
DATA_DIR = '../data/'
fn_uid_sids = 'uid_sids.txt'

In [325]:
idx2uid, uid2idx, idx2sid, sid2idx, uid_sids_dict, n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te =\
    preprocess.preprocess_data(data_dir=DATA_DIR, fn_uid_sids=fn_uid_sids)

filter out items watched by and users watches less than:5
../data/data_train.txt ../data/data_dev.txt ../data/data_test.txt
n_users:32296
n_items:50000


In [326]:
### train
trainer = imp.reload(trainer)
metric = imp.reload(metric)

In [327]:
vae = trainer.train(n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te)

INFO:tensorflow:Scale of 0 disables regularizer.
log directory: ./log/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
chkpt directory: ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
epoch:0
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.10310717673875107 best_ndcg:-inf
epoch:1
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_inde

  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.16051932273991196 best_ndcg:0.1590866131167707
epoch:16
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.16406032285561936 best_ndcg:0.16051932273991196
epoch:17
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_i

  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.22520489885991146 best_ndcg:0.22111990134309908
epoch:33
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.2289965605040167 best_ndcg:0.22520489885991146
epoch:34
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end

  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.26531229441698057 best_ndcg:0.26375290105428695
epoch:49
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.2691268022422741 best_ndcg:0.26531229441698057
epoch:50
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end

  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.2978441797166204 best_ndcg:0.2964457313419415
epoch:65
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.29838508421141935 best_ndcg:0.2978441797166204
epoch:66
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_i

...

  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.3375457760194943 best_ndcg:0.33662570781076484
epoch:130
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.33694707091668613 best_ndcg:0.3375457760194943
epoch:131
  batch_num:0 start_index:0 e

  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.34038315793612234 best_ndcg:0.33971191301353065
epoch:146
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.3392462667241921 best_ndcg:0.34038315793612234
epoch:147
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 e

  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.34161243020123305 best_ndcg:0.3421862333399157
epoch:162
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.34139940278140635 best_ndcg:0.3421862333399157
epoch:163
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 en

  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.34482296272020363 best_ndcg:0.3447456907226203
epoch:178
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.3448805871713761 best_ndcg:0.34482296272020363
epoch:179
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 en

  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.3461332484986397 best_ndcg:0.34738493818153954
epoch:194
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 end_index:25000 N:25838
  batch_num:5 start_index:25000 end_index:25838 N:25838
  batch_num:0 start_index:0 end_index:2000 N_vad:3229
  batch_num:1 start_index:2000 end_index:3229 N_vad:3229
  cur_ndcg:0.34626466971610714 best_ndcg:0.34738493818153954
epoch:195
  batch_num:0 start_index:0 end_index:5000 N:25838
  batch_num:1 start_index:5000 end_index:10000 N:25838
  batch_num:2 start_index:10000 end_index:15000 N:25838
  batch_num:3 start_index:15000 end_index:20000 N:25838
  batch_num:4 start_index:20000 e

In [329]:
### evaluate

In [330]:
evaluator.evaluate(n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te)

INFO:tensorflow:Scale of 0 disables regularizer.
chkpt directory: ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
Test NDCG@100=0.34018 (0.00395)
Test Recall@20=0.29627 (0.00433)
Test Recall@50=0.35474 (0.00434)


In [331]:
### predict

In [332]:
base_sididx_arr = None
uid = '3025'
if uid not in uid2idx:
    print('uid not in uid2idx')
    print('len(uid2idx):{}'.format(len(uid2idx)))
else:
    raw_sids = uid_sids_dict[uid]
    sids = []
    for sid in raw_sids:
        if sid in sid2idx:
            sids.append(sid)
    base_sid_arr = sids
    base_sididx_arr = [sid2idx[sid] for sid in base_sid_arr]
print(base_sid_arr)

['M1014282796', 'M1370493287', 'M1052970948', 'M1377166928', 'M1056570414', 'M997731459', 'M1027597021', 'M1271200417', 'S8430269609739938089', 'M1289716254', 'M994223815', 'M1016280242', 'M1332639531', 'M1317673681', 'M1307290396', 'M1144999190', 'M1014804743', 'M991053430', 'M1005075935', 'M975203415', 'M1054258194', 'M1353502577', 'M1313313268', 'M1009485140', 'M991407576', 'M964643352', 'M992222638', 'M1022000981', 'M1024089935', 'M1101947456', 'M1028818936', 'M1312729793', 'M1049138943', 'M1104060320', 'M1307740806', 'M995355801', 'M1024611341', 'M1029294466', 'M1086568183', 'M1051610271', 'R1', 'M1015730902', 'M1050731970', 'M1028669731', 'M1098050321', 'M1088339200', 'M979770756', 'M964989942', 'S8595251796415912536', 'M1054824855', 'M1015325479', 'M1112234209', 'M1091440579', 'M1146305378', 'M1001442387', 'M1004610622', 'M1025904638', 'M1309335566', 'M1313194916', 'S7093446576167723636', 'M1091007487', 'M1370385124', 'M1088921323', 'M1012301515', 'M1148928911', 'S95342954830238

In [333]:
count = 30
input_sididx_arr = base_sididx_arr[:count]
print([idx2sid[idx] for idx in input_sididx_arr])

['M1014282796', 'M1370493287', 'M1052970948', 'M1377166928', 'M1056570414', 'M997731459', 'M1027597021', 'M1271200417', 'S8430269609739938089', 'M1289716254', 'M994223815', 'M1016280242', 'M1332639531', 'M1317673681', 'M1307290396', 'M1144999190', 'M1014804743', 'M991053430', 'M1005075935', 'M975203415', 'M1054258194', 'M1353502577', 'M1313313268', 'M1009485140', 'M991407576', 'M964643352', 'M992222638', 'M1022000981', 'M1024089935', 'M1101947456']


In [334]:
scores = evaluator.predict(input_sididx_arr, n_items)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model


In [335]:
scores = scores.ravel()
topk = 30
indices = np.argpartition(-scores, topk)[:topk]
print([idx2sid[sididx] for sididx in indices])
#print(scores[indices][:topk])

['M1008401098', 'M1093750375', 'M1050731970', 'M1104060320', 'M1051610271', 'M1058761068', 'M964989942', 'M1091440579', 'M1105736573', 'M1283457844', 'M1028818936', 'M1146305378', 'M1026810877', 'M1345646809', 'M1129554731', 'M981280476', 'M1004610622', 'M1001442387', 'M987285300', 'M1312729793', 'M1015325479', 'M993189565', 'M1088339200', 'M1085621244', 'M1098183980', 'M1012301515', 'M1012282237', 'M978045018', 'M1313194916', 'M1133617907']


In [336]:
## output tag
import sid.resource_tag as tag
#tag.get("U57778464659")[:1]

In [337]:
arr = [idx2sid[idx] for idx in input_sididx_arr]
[(sid, tag.get(sid)) for sid in arr]

get() error M1014282796
get() error M1056570414
get() error M1271200417
get() error M1024089935
get() error M1101947456


[('M1014282796', (None, None)),
 ('M1370493287',
  ('后弦^sara,你还欠我一个拥抱', '回忆,中文,华语,对唱,经典,情歌,精选,后弦,sara,很有爱,潮女密咒 ep')),
 ('M1052970948',
  ('armik,Guitar Romance', '吉他大师,纯音乐,纯音,guitar,精选,弗拉门戈,吉他,armik,gypsy flame')),
 ('M1377166928',
  ('tobu,Colors', '电音,bgm,游戏,纯音,纯音乐,抖腿,节奏,tobu,colors,power mixes for clubs')),
 ('M1056570414', (None, None)),
 ('M997731459',
  ('鹏泊,啷个哩个啷', 'bgm,国语,故事,好听,华语,中文,民谣,鹏泊,啷个哩个啷,觅乐行动·荣耀制噪者校园音乐节南京站')),
 ('M1027597021',
  ('antonio de lucena,Bésame Mucho',
   'guitar,弗拉门戈,吉他,精选,弗拉门戈吉他,纯音,纯音乐,antonio de lucena,100 classical spanish guitar,spanish guitar: music of spain,guitarra latina,traditional hits instrumental: spanish guitar,clásicos instrumentales: guitarra espaola,guitarras y castauelas,100% guitarra espaola. latin music with spanish guitar. música ambienter instrumental,a ritmo de guitarra espaola vol. 1,alma de guitarra,grandes clásicos de guitarra espaola,en privado... guitarra vol. 2,spanish classical guitar,spanish guitar 1,guitarra espaola,música rela

In [338]:
arr = [idx2sid[sididx] for sididx in indices]
[(sid, tag.get(sid)) for sid in arr]

get() error M1093750375
get() error M1098183980
get() error M1133617907


[('M1008401098',
  ('林海,远方的寂静', '纯音,钢琴,纯音乐,轻音,精选,轻音乐,原创,林海,远方的寂静,风潮音乐风景 20周年庆生专辑')),
 ('M1093750375', (None, None)),
 ('M1050731970',
  ('incendio,Ledges Road', '忧伤,弗拉门戈吉他,吉他,弗拉门戈,incendio,seduction')),
 ('M1104060320',
  ('paul mauriat,Invitation to the Dance (Live in Osaka)',
   '精选,轻音,古典,交响,轻音乐,生活,classic,paul mauriat,paul mauriat. grandes exitos 2,romantic,sayonara concert,classic&new')),
 ('M1051610271', ('张宇桦,风中的蒲公英(钢琴曲)', '世界,好听,轻音乐,轻音,钢琴,纯音,纯音乐,张宇桦,星空下的钢琴曲')),
 ('M1058761068',
  ('douglas jimerson,The Stars and Stripes Forever',
   '古典,douglas jimerson,america,god bless america')),
 ('M964989942',
  ('李闰珉,When The Love Falls',
   '轻音乐,精选,轻音,背景,钢琴,纯音,纯音乐,李闰珉,yiruma the best winter,the very best of yiruma: yiruma&piano,first love,prenatal education piano music,when love falls,最新热歌慢摇103,missing you,piano museum,live at hoam art hall')),
 ('M1091440579', ('张宇桦,被遗忘的89键(钢琴曲)', '纯音,钢琴,纯音乐,轻音,轻音乐,世界,好听,张宇桦,星空下的钢琴曲')),
 ('M1105736573', ('张宇桦,星星的夜空(钢琴曲)', '纯音乐,世界,好听,轻音乐,轻音,纯音,钢琴,张宇桦,星空下的

In [339]:
print('done')

done


In [340]:
### output user sample

In [273]:
def dump_file(fd, arr_info):
    for sid, info in arr_info:
        fd.write('{},{},{}\n'.format(sid, info[0], info[1]))

In [268]:
import os
import random
SAMPLE_DIR = '../data/sample/'
SAMPLE_NUM = 100
MAX_INPUT_SID_NUM = 1000
TOP_K = 100
if not os.path.exists(SAMPLE_DIR):
    os.makedirs(SAMPLE_DIR)

In [270]:
import pandas as pd
df = pd.read_excel("/home/dev/notebook/top_cuid_20180601-20180630.xls")
excel_uids = df["dumi_id"].tolist()
print(excel_uids[:5])

[168273265, 105358163, 170477245, 41004486, 174068888]


In [271]:
# get real history
from his_session_util.util import get_user_history
import json
def get_satisfied_sids(uid):
    his = get_user_history(
        uid_to_namelist={uid: ['latest_accessed_resources']}).json()
    his = json.loads(his["data"][0]['value'])['resource']
    his_satisfied = []
    for elem in his:
        if int(elem['time_elapse']) > 60000:
            if elem['sid'][0] == 'M' or elem['sid'][0] == 'U' or elem['sid'][0] == 'S':
                his_satisfied.append(elem)

    his_data = []
    for item in his_satisfied:
        if item is not None:
            his_data.append(item)
    return pd.DataFrame(his_data)["sid"].tolist()

In [274]:
#sample_uids = random.sample(list(uid2idx.keys()), SAMPLE_NUM)
sample_uids = excel_uids
cur_idx = 0
for uid in sample_uids:
    uid = str(uid)
    cur_idx += 1
    print('{}:{}'.format(cur_idx, uid))
    #raw_sids = uid_sids_dict[uid] # data
    raw_sids = get_satisfied_sids(uid) # real history
    sids = []
    for sid in raw_sids:
        if sid in sid2idx:
            sids.append(sid)
    base_sid_arr = sids
    base_sididx_arr = [sid2idx[sid] for sid in base_sid_arr]
    input_sididx_arr = base_sididx_arr[:MAX_INPUT_SID_NUM]
    scores = evaluator.predict(input_sididx_arr, n_items)
    scores = scores.ravel()
    indices = np.argpartition(-scores, topk)[:TOP_K]
    arr_input = [idx2sid[idx] for idx in input_sididx_arr]
    arr_info_input = [(sid, tag.get(sid)) for sid in arr_input]
    arr_predict = [idx2sid[sididx] for sididx in indices]
    arr_info_predict = [(sid, tag.get(sid)) for sid in arr_predict]
    fd_in = open(SAMPLE_DIR + 'uid' + uid + '_history.csv', 'w', encoding='utf-8')
    fd_out = open(SAMPLE_DIR + 'uid' + uid + '_recommend.csv', 'w', encoding='utf-8')
    dump_file(fd_in, arr_info_input)
    dump_file(fd_out, arr_info_predict)
    fd_in.close()
    fd_out.close()

1:168273265
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error U85081034548
get() error M1327802868
get() error U57735790532
_get() error R1
get() error M1372575997
get() error M1180804765
get() error M1055706684
get() error M1255477721
get() error M1264926014
2:105358163
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1013472044
get() error M1055706684
get() error M1001165755
3:170477245
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1273609967
get() error M1372575997
get() error M1327802868
get() error M1264926014
get() error M1055706684
get() error M1332662583
get() error U57735790538
get() error M976850328
get() error M1334448000
get() error U577357

get() error M1262015128
get() error M1017106267
get() error M1026596149
get() error M1375763542
get() error M1372575997
get() error M1055706684
get() error M1382474501
get() error M1001165755
get() error M1018936430
get() error M1334448000
15:169778976
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1050044835
get() error M1050044835
get() error M1322433576
get() error M1187309672
get() error M1322433576
get() error M1322433576
get() error M1180804765
get() error M1055706684
_get() error R1
16:171756316
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1327802868
get() error M984599676
get() error U57738354400
get() error M1005059134
get() error U57782658148
get() error M1093750375
get() error M1055706684
get() error U57803265120
17:161736276
INFO:tensorflow:Scale

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1384905341
get() error M1142047002
get() error M995016453
get() error M1264926014
get() error U57776233667
get() error U57783357871
get() error U57779025656
get() error U57780049969
get() error U57776233535
get() error U57803351271
get() error U57783357832
get() error U57782658148
get() error U57803351319
get() error U57802747206
get() error U57802747290
get() error U57783357799
get() error U56856356133
get() error M1055706684
get() error U57780049909
get() error U57783357841
get() error U57779610896
get() error U57782770735
get() error M1374283756
33:42130650
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1139340519
get() error M1172300173
get() error M1141662045
get() error M1180560439
get() error M1154833122
ge

get() error M1382493644
get() error M1268744044
get() error U85081034548
get() error U74051734408
get() error M1382474501
get() error M1113288781
_get() error R168
get() error M1013484143
_get() error R1
_get() error J56490412077
get() error U57735790532
51:170636003
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1026596149
get() error M1017106267
get() error M1026596149
get() error M1026596149
get() error M1026596149
get() error M1026596149
get() error M1026596149
get() error M1026596149
get() error M1026596149
get() error M1017106267
get() error M1017106267
get() error M1026596149
get() error M986407020
get() error M1017106267
get() error U57783297181
get() error U57783297181
get() error U57803432616
get() error U57779540270
get() error U57779540225
get() error U57783297181
get() error M1327802868
_get() error R1098
get() error M1329121692
get() error M998809814
get() er

get() error M1290452277
get() error M1001563042
get() error M1055706684
get() error M1093750375
get() error M1353490862
get() error M996855871
get() error M1327802877
66:173352952
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1093750375
get() error M1148928911
get() error M1149644739
get() error M1178704027
get() error U57782658148
get() error M1327802868
get() error M1055706684
_get() error R567
get() error U57803265117
get() error U57803265120
67:162321656
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1322432349
get() error M1089364591
get() error M1002987961
get() error M1019879818
get() error M1043075507
get() error M1026707758
get() error M1308522001
get() error M1050197157
get() error M993918944
get() error M1055706684
68:170331242
INFO:tensorflow:Scal

...

get() error U85079119190
get() error U85079072647
get() error U85079072647
get() error U85078373995
get() error U85077360252
get() error U85076649609
get() error U85071761171
get() error U85071761171
get() error U85071241952
get() error U85070634308
get() error U85069904684
get() error U85069977160
get() error U85068680099
get() error U85068292607
get() error U85069010068
get() error U85063997023
get() error U85063042878
get() error U85062353496
get() error U85061819664
get() error U85061298372
get() error U85060634774
get() error U85060634774
get() error U85060138589
get() error U85059674699
get() error U85059674699
get() error U85059799000
get() error U85058630945
get() error U85058088221
get() error U85057547978
get() error U85057475962
get() error U85056369945
get() error U85056276754
get() error U57782658148
get() error U57740166409
849:159056864
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200

get() error U62268829189
get() error M1367500918
get() error U85071241952
get() error U57803408118
get() error U85077360252
get() error M1382474501
857:19408959
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M987437445
get() error M1240645144
get() error M987437445
get() error M997690844
get() error M1055706684
get() error M1372575997
get() error M1001165755
_get() error R1
get() error M1180804765
_get() error R66
858:166989443
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1001474748
get() error M998236285
get() error M1034143070
get() error M1005890636
get() error M996018378
get() error M1029116029
get() error M1029116029
get() error M1029116029
get() error M1029116029
get() error M1029116029
get() error M1050255138
get() error M1029116029
get() error M102911

874:50471520
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error U57779595146
get() error U57779595158
get() error U57803336826
get() error U57803336814
get() error U57778471835
get() error M1382474501
get() error M1001165755
get() error M1372575997
get() error U57782658148
get() error M1055706684
get() error M1334448000
get() error M1018936430
875:52728620
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error U85081034548
get() error U85079858819
get() error U85079119190
get() error U85079072647
get() error U85078373995
get() error U85077360252
get() error U85076649609
get() error U85071761171
get() error U85071761171
get() error U85071241952
get() error U85070634308
get() error U85069904684
get() error U85069977160
get() error U85068680099
get() error U85068292607
get() e

get() error M1221161147
get() error M983910816
890:49571036
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M996855871
get() error M1093750375
get() error M1001563042
_get() error R1
get() error M1055706684
get() error M1367191144
_get() error N85065306053
_get() error R21
891:25889630
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1093750375
get() error M1269757207
get() error M1269774937
get() error M1272803891
get() error M1317434141
get() error M1055706684
get() error M974714007
_get() error R1
get() error M1332662583
get() error M992248861
892:157653235
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
get() error M1377386843
get() error M1016012686
get() erro

KeyError: 'sid'

In [380]:
# save model
!rm -r '../model/'
SIGNATURE_NAME = "serving_default"

In [381]:
model = imp.reload(model)
total_anneal_steps = 200000
anneal_cap = 0.2
p_dims = [200, 600, n_items]
tf.reset_default_graph()
vae = model.MultiVAE(p_dims, lam=0.0)
saver, logits_var, _, _, _ = vae.build_graph()
arch_str = "I-%s-I" % ('-'.join([str(d) for d in vae.dims[1:-1]]))
chkpt_dir = './chkpt/VAE_anneal{}K_cap{:1.1E}/{}'.format(
    total_anneal_steps/1000, anneal_cap, arch_str)

X = sparse.csr_matrix((np.ones_like(base_sididx_arr), (np.zeros_like(base_sididx_arr), base_sididx_arr)), shape=(1, n_items), dtype=np.int16)
if sparse.isspmatrix(X):
    X = X.toarray()
X = X.astype('float32')

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    saver.restore(sess, '{}/model'.format(chkpt_dir))
    pred_val = sess.run(vae.logits, feed_dict={vae.input_ph: X, vae.is_training_ph: 0})
    pred_val[X.nonzero()] = -np.inf
    scores = pred_val
    scores = scores.ravel()
    topk = 30
    indices = np.argpartition(-scores, topk)[:topk]
    print([idx2sid[sididx] for sididx in indices])
    builder = tf.saved_model.builder.SavedModelBuilder(MODEL_DIR)
    builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING], signature_def_map= {
            SIGNATURE_NAME: tf.saved_model.signature_def_utils.build_signature_def(
                inputs= {"X": tf.saved_model.utils.build_tensor_info(vae.input_ph)},
                outputs= {"y_values": tf.saved_model.utils.build_tensor_info(vae.logits_top_values),
                          "y_indices": tf.saved_model.utils.build_tensor_info(vae.logits_top_indices)},
                method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
            })
    #builder.save(as_text=True)
    builder.save()

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
['M1028003932', 'M993189565', 'M978045018', 'M1037043782', 'M1030033171', 'M970165866', 'M1010448014', 'M998753215', 'M1094487754', 'M1283457844', 'M1099177922', 'M1010227700', 'M1098183980', 'M1053971508', 'M1057638240', 'M1042166264', 'M1091332953', 'M1052817711', 'M1133617907', 'M1030941281', 'M1025634551', 'M1097844527', 'M1079476191', 'M997374012', 'M1278825539', 'M1008401098', 'M1026810877', 'M1304516818', 'M1329836160', 'M1089608533']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../model/saved_model.pb'


In [382]:
# load model
np.shape(X)

(1, 50000)

In [383]:
pred_val = None
with tf.Session(graph=tf.Graph()) as sess:
    meta_graph_def = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], MODEL_DIR)
    signature = meta_graph_def.signature_def
    
    X_tensor_name = signature[SIGNATURE_NAME].inputs["X"].name
    y_values_tensor_name = signature[SIGNATURE_NAME].outputs["y_values"].name
    y_indices_tensor_name = signature[SIGNATURE_NAME].outputs["y_indices"].name
    
    X_t = sess.graph.get_tensor_by_name(X_tensor_name)
    y_val_t = sess.graph.get_tensor_by_name(y_values_tensor_name)
    y_ind_t = sess.graph.get_tensor_by_name(y_indices_tensor_name)

    print(sess.run([y_val_t, y_ind_t], feed_dict={X_t: X}))

INFO:tensorflow:Restoring parameters from b'../model/variables/variables'
[array([[11.173062 , 11.023243 , 10.987673 , 10.915925 , 10.864816 ,
        10.601925 , 10.58757  , 10.545959 , 10.531741 , 10.524103 ,
        10.465421 , 10.426731 , 10.400718 , 10.256097 , 10.12566  ,
        10.093816 ,  9.997297 ,  9.944538 ,  9.893774 ,  9.877995 ,
         9.870592 ,  9.846984 ,  9.843458 ,  9.771355 ,  9.708678 ,
         9.456582 ,  9.453215 ,  9.410031 ,  9.362083 ,  9.359968 ,
         9.357004 ,  9.346919 ,  9.32815  ,  9.315765 ,  9.261443 ,
         9.261258 ,  9.257811 ,  9.216233 ,  9.214954 ,  9.11876  ,
         9.109643 ,  9.0889225,  9.060694 ,  8.872055 ,  8.840444 ,
         8.710972 ,  8.650212 ,  8.570688 ,  8.555746 ,  8.555513 ,
         8.552863 ,  8.54456  ,  8.537918 ,  8.501936 ,  8.500536 ,
         8.4628935,  8.401162 ,  8.396132 ,  8.360429 ,  8.343392 ,
         8.320563 ,  8.298549 ,  8.296173 ,  8.238878 ,  8.220858 ,
         8.20595  ,  8.167709 ,  8.12115 

In [375]:
scores = pred_val
scores = scores.ravel()
topk = 30
indices = np.argpartition(-scores, topk)[:topk]

arr = [idx2sid[sididx] for sididx in indices]
[(sid, tag.get(sid)) for sid in arr]

get() error M1099177922
get() error M1010227700
get() error M1098183980
get() error M1042166264
get() error M1052817711
get() error M1030941281
get() error M1304516818


[('M1028003932',
  ('claude debussy,Reverie',
   "classic,classical,钢琴,纯音,piano,精选,古典,claude debussy,claude debussy plays his finest works,piano works: two arabesques / preludes / claire de lune / l'isle joyeuse / children's corner,clair de lune,100 best piano classics,opus")),
 ('M993189565', ('林海,欢沁', '轻音乐,古风,精选,轻音,纯音乐,纯音,中国,林海,林海影视配乐精选,琵琶相')),
 ('M978045018', ('林海,踏古', '纯音乐,古风,轻音乐,轻音,精选,中国,纯音,林海,2005风潮音乐纪行,琵琶相')),
 ('M1037043782',
  ('noicybino,Fell for U', '纯音,钢琴,安静,轻音乐,纯音乐,轻音,bgm,noicybino,fell for u')),
 ('M1030033171', ('bandari,Bandar', 'bandari,khaterkhah')),
 ('M970165866',
  ('antonio de lucena,Romance Anónimo',
   '纯音,轻音,弗拉门戈,吉他,guitar,古典,纯音乐,antonio de lucena,100 classical spanish guitar,guitarra instrumental espaola. ambiente musical,classical guitar,grandes clásicos de guitarra espaola,guitarra espaola romantica. música pop instrumental para ambiente musical. background music,typical spanish guitar,guitar strings vol. 2,corazón de guitarra,guitarra latina,the best spanis

In [346]:
tag.get('M1012282237')

('林海,琵琶语', '精选,轻音,原创,轻音乐,纯音,纯音乐,古风,林海,琵琶相,林海影视配乐精选,hifi极品测试vi 吹拉弹器乐大全')